In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler

In [3]:
df = pd.read_csv('../data/preprocessed/macro_finance_data_2014_2023.csv')
df

,Date,WTI Price,Natural Gas Price,Aluminum Price,Copper Price,Gold Price,Platinum Price,Silver Price,Lumber Price,Interest_Rate,...,NASDAQ,Nikkei 225,Russell 2000,S&P 500,TSX,CADJPY,EURCAD,GBPCAD,USDCAD,10 YR Bond Yield
0,2014-01-01,98.70,4.232,1840.25,3.3815,1225.2,1377.05,20.128,353.3,1.0,...,4143.07,15908.88,1150.72,1831.98,13594.19,98.88,1.4640,1.7634,1.0644,2.74
1,2014-01-02,95.44,4.321,1840.25,3.3815,1225.2,1402.30,20.128,353.3,1.0,...,4143.07,15908.88,1150.72,1831.98,13594.19,98.18,1.4589,1.7558,1.0671,2.74
2,2014-01-03,93.96,4.304,1840.25,3.3550,1238.6,1412.45,20.211,356.8,1.0,...,4131.91,15908.88,1156.09,1831.37,13548.86,98.59,1.4452,1.7460,1.0635,2.75
3,2014-01-04,93.96,4.304,1840.25,3.3550,1238.6,1412.45,20.211,356.8,1.0,...,4131.91,15908.88,1156.09,1831.37,13548.86,98.59,1.4452,1.7460,1.0635,2.75
4,2014-01-05,93.96,4.304,1840.25,3.3550,1238.6,1407.30,20.211,356.8,1.0,...,4131.91,15908.88,1156.09,1831.37,13548.86,98.59,1.4452,1.7460,1.0635,2.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3258,2024-03-27,76.49,1.603,2180.00,3.8800,2049.4,909.60,22.982,563.5,5.0,...,15996.82,39098.68,2016.69,5088.80,21413.15,111.42,1.4610,1.7119,1.3505,3.10
3259,2024-03-28,76.49,1.603,2180.00,3.8800,2049.4,909.60,22.982,563.5,5.0,...,15996.82,39098.68,2016.69,5088.80,21413.15,111.42,1.4610,1.7119,1.3505,3.10
3260,2024-03-29,76.49,1.603,2180.00,3.8800,2049.4,909.60,22.982,563.5,5.0,...,15996.82,39098.68,2016.69,5088.80,21413.15,111.42,1.4610,1.7119,1.3505,3.10
3261,2024-03-30,76.49,1.603,2180.00,3.8800,2049.4,909.60,22.982,563.5,5.0,...,15996.82,39098.68,2016.69,5088.80,21413.15,111.42,1.4610,1.7119,1.3505,3.10


In [4]:
df['Index Up'] = df['TSX'] > df['TSX'].shift(1)
df.head(10)

,Date,WTI Price,Natural Gas Price,Aluminum Price,Copper Price,Gold Price,Platinum Price,Silver Price,Lumber Price,Interest_Rate,...,Nikkei 225,Russell 2000,S&P 500,TSX,CADJPY,EURCAD,GBPCAD,USDCAD,10 YR Bond Yield,Index Up
0,2014-01-01,98.70,4.232,1840.25,3.3815,1225.2,1377.05,20.128,353.3,1.0,...,15908.88,1150.72,1831.98,13594.19,98.88,1.4640,1.7634,1.0644,2.74,False
1,2014-01-02,95.44,4.321,1840.25,3.3815,1225.2,1402.30,20.128,353.3,1.0,...,15908.88,1150.72,1831.98,13594.19,98.18,1.4589,1.7558,1.0671,2.74,False
2,2014-01-03,93.96,4.304,1840.25,3.3550,1238.6,1412.45,20.211,356.8,1.0,...,15908.88,1156.09,1831.37,13548.86,98.59,1.4452,1.7460,1.0635,2.75,False
3,2014-01-04,93.96,4.304,1840.25,3.3550,1238.6,1412.45,20.211,356.8,1.0,...,15908.88,1156.09,1831.37,13548.86,98.59,1.4452,1.7460,1.0635,2.75,False
4,2014-01-05,93.96,4.304,1840.25,3.3550,1238.6,1407.30,20.211,356.8,1.0,...,15908.88,1156.09,1831.37,13548.86,98.59,1.4452,1.7460,1.0635,2.75,False
5,2014-01-06,93.43,4.306,1840.25,3.3595,1238.0,1415.60,20.103,352.0,1.0,...,15908.88,1147.16,1826.77,13495.54,97.81,1.4522,1.7481,1.0655,2.72,False
6,2014-01-07,93.67,4.299,1840.25,3.3595,1229.6,1413.65,19.787,354.2,1.0,...,15814.37,1157.63,1837.88,13596.93,97.18,1.4658,1.7658,1.0766,2.67,True
7,2014-01-08,92.33,4.216,1840.25,3.3425,1225.5,1416.65,19.539,356.0,1.0,...,16121.45,1157.46,1837.49,13614.63,96.91,1.4688,1.7795,1.0819,2.72,True
8,2014-01-09,91.66,4.005,1840.25,3.2990,1229.4,1418.90,19.683,365.5,1.0,...,15880.33,1158.35,1838.13,13629.41,96.70,1.4751,1.7866,1.0840,2.68,True
9,2014-01-10,92.72,4.053,1840.25,3.3415,1246.9,1437.35,20.223,362.2,1.0,...,15912.06,1164.53,1842.37,13747.52,95.60,1.4892,1.7956,1.0895,2.56,True


### For EDA set the date as the index to leverage time series specific methods for understanding the data

In [5]:
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)


In [6]:
df['Year'] = df.index.year
df['Month'] = df.index.month
df['Day'] = df.index.day
df['DayofWeek'] = df.index.dayofweek
df

,WTI Price,Natural Gas Price,Aluminum Price,Copper Price,Gold Price,Platinum Price,Silver Price,Lumber Price,Interest_Rate,GDP,...,CADJPY,EURCAD,GBPCAD,USDCAD,10 YR Bond Yield,Index Up,Year,Month,Day,DayofWeek
Date,,,,,,,,,,,,,,,,,,,,,
2014-01-01,98.70,4.232,1840.25,3.3815,1225.2,1377.05,20.128,353.3,1.0,1863035.0,...,98.88,1.4640,1.7634,1.0644,2.74,False,2014,1,1,2
2014-01-02,95.44,4.321,1840.25,3.3815,1225.2,1402.30,20.128,353.3,1.0,1863035.0,...,98.18,1.4589,1.7558,1.0671,2.74,False,2014,1,2,3
2014-01-03,93.96,4.304,1840.25,3.3550,1238.6,1412.45,20.211,356.8,1.0,1863035.0,...,98.59,1.4452,1.7460,1.0635,2.75,False,2014,1,3,4
2014-01-04,93.96,4.304,1840.25,3.3550,1238.6,1412.45,20.211,356.8,1.0,1863035.0,...,98.59,1.4452,1.7460,1.0635,2.75,False,2014,1,4,5
2014-01-05,93.96,4.304,1840.25,3.3550,1238.6,1407.30,20.211,356.8,1.0,1863035.0,...,98.59,1.4452,1.7460,1.0635,2.75,False,2014,1,5,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-27,76.49,1.603,2180.00,3.8800,2049.4,909.60,22.982,563.5,5.0,2218374.0,...,111.42,1.4610,1.7119,1.3505,3.10,False,2024,3,27,2
2024-03-28,76.49,1.603,2180.00,3.8800,2049.4,909.60,22.982,563.5,5.0,2218374.0,...,111.42,1.4610,1.7119,1.3505,3.10,False,2024,3,28,3
2024-03-29,76.49,1.603,2180.00,3.8800,2049.4,909.60,22.982,563.5,5.0,2218374.0,...,111.42,1.4610,1.7119,1.3505,3.10,False,2024,3,29,4


## Train-Test-Split

In [7]:
X = df.drop(columns=['Index Up'])
y = df['Index Up']

In [8]:
tss = TimeSeriesSplit(n_splits=3)

In [9]:
for train_index, test_index in tss.split(X):
    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index,:]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

In [12]:
X_train

,WTI Price,Natural Gas Price,Aluminum Price,Copper Price,Gold Price,Platinum Price,Silver Price,Lumber Price,Interest_Rate,GDP,...,TSX,CADJPY,EURCAD,GBPCAD,USDCAD,10 YR Bond Yield,Year,Month,Day,DayofWeek
Date,,,,,,,,,,,,,,,,,,,,,
2014-01-01,98.70,4.232,1840.25,3.3815,1225.2,1377.05,20.128,353.3,1.00,1863035.0,...,13594.19,98.88,1.4640,1.7634,1.0644,2.74,2014,1,1,2
2014-01-02,95.44,4.321,1840.25,3.3815,1225.2,1402.30,20.128,353.3,1.00,1863035.0,...,13594.19,98.18,1.4589,1.7558,1.0671,2.74,2014,1,2,3
2014-01-03,93.96,4.304,1840.25,3.3550,1238.6,1412.45,20.211,356.8,1.00,1863035.0,...,13548.86,98.59,1.4452,1.7460,1.0635,2.75,2014,1,3,4
2014-01-04,93.96,4.304,1840.25,3.3550,1238.6,1412.45,20.211,356.8,1.00,1863035.0,...,13548.86,98.59,1.4452,1.7460,1.0635,2.75,2014,1,4,5
2014-01-05,93.96,4.304,1840.25,3.3550,1238.6,1407.30,20.211,356.8,1.00,1863035.0,...,13548.86,98.59,1.4452,1.7460,1.0635,2.75,2014,1,5,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-29,74.83,5.477,2911.50,4.1990,1722.9,947.00,21.485,606.5,0.25,2105441.0,...,20158.14,87.76,1.4788,1.7118,1.2754,1.51,2021,9,29,2
2021-09-30,75.03,5.867,2858.50,4.0890,1757.0,962.40,22.047,627.5,0.25,2111870.0,...,20070.25,87.74,1.4682,1.7080,1.2678,1.51,2021,9,30,3
2021-10-01,75.88,5.619,2857.00,4.1885,1758.4,973.60,22.536,625.1,0.25,2111870.0,...,20150.87,87.78,1.4663,1.7130,1.2647,1.47,2021,10,1,4


In [13]:
scaler = StandardScaler()

X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns) # maintain column names
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)

In [14]:
X_train_scaled.to_csv('../data/processed/X_train_scaled.csv', index=False)
X_test_scaled.to_csv('../data/processed/X_test_scaled.csv', index=False)
y_train.to_csv('../data/processed/y_train.csv', index=False)
y_test.to_csv('../data/processed/y_test.csv', index=False)